# Server Log Data Extraction and User History Database Update

### Author:  Aditya Balasubramaniam    
### Last Updated:  August 2025

### Project Overview

**Project Title:** Server Log Data Extraction and User History Database Update  
**Technologies:**  Python, SQL, SQLite, MongoDB, Regex  
**Dataset:**  mbox.txt (Email server log file)

### Problem Statement

The task is to fetch data from a server log file, extract all email addresses along with their corresponding dates, and upload this data into a user history database. The goal is to ensure the extracted data is clean, accurate, and accessible for further analysis and historical tracking.

### Data Pipeline Overview

```
mbox.txt → Data Extraction → MongoDB → SQLite → Analysis & Insights
```

## Step 1: Import Required Libraries and Setup Configuration

In [2]:
# Import all required libraries

import os
import re
from datetime import datetime
from pymongo import MongoClient
import sqlite3
import pandas as pd


# Import database configuration
from db_config import MONGO_CONFIG, SQLITE_CONFIG, GOOGLE_DRIVE_CONFIG, USER_HISTORY_SCHEMA, ANALYSIS_QUERIES

# Install gdown if not available
try:
    import gdown
except ImportError:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'gdown'])
    import gdown

print("All libraries imported successfully!")

## Step 2: Download Dataset from Google Drive

In [5]:
# Google Drive file ID and output filename
file_id = '13qr0NacvjeqC1jPESa8A3ViFY8XtEQAb'
output = 'mbox.txt'

# Download the file if it does not exist
if not os.path.exists(output):
    url = f'https://drive.google.com/uc?id={file_id}'
    print(f"Downloading dataset from {url} ...")
    gdown.download(url, output, quiet=False)
else:
    print(f"Dataset '{output}' already exists.")

Dataset 'mbox.txt' already exists.


## Step 3: Data Extraction - (Transformation)-Extract Email Addresses and Dates

In [14]:
# Step 2: Extract email addresses and dates from mbox.txt 
# We will extract lines starting with 'From ' (not 'From:') and parse the email and date from those lines.

def extract_emails_and_dates(filename):
    pattern = re.compile(r"^From ([^ ]+) (.*)")
    extracted = []
    with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            match = pattern.match(line)
            if match:
                email = match.group(1)
                date_str = match.group(2)
                # Try to parse the date string into a standard format
                try:
                    # Example: 'Sat Jan  5 09:14:16 2008'
                    dt = datetime.strptime(date_str, '%a %b %d %H:%M:%S %Y')
                except ValueError:
                    # Handle single-digit days with extra space
                    try:
                        dt = datetime.strptime(date_str, '%a %b %d %H:%M:%S %Y')
                    except Exception:
                        continue  # Skip if date can't be parsed
                extracted.append({
                    'email': email,
                    'date': dt.strftime('%Y-%m-%d %H:%M:%S')
                })
    return extracted

In [15]:
# Extracted data will be a list of dicts with 'email' and 'date' keys
extracted_data = extract_emails_and_dates('mbox.txt')
print(f"Extracted {len(extracted_data)} email-date pairs.")

# Preview first 3 entries
print(extracted_data[:3])

Extracted 1797 email-date pairs.
[{'email': 'stephen.marquard@uct.ac.za', 'date': '2008-01-05 09:14:16'}, {'email': 'louis@media.berkeley.edu', 'date': '2008-01-04 18:10:48'}, {'email': 'zqian@umich.edu', 'date': '2008-01-04 16:10:39'}]


## Step 5: Save Data to MongoDB



In [16]:
# Connect to local MongoDB and insert the extracted data into 'user_history' collection

def save_to_mongodb(data, db_name='logdata', collection_name='user_history'):
    client = MongoClient('mongodb://localhost:27017/')
    db = client[db_name]
    collection = db[collection_name]
    # Remove existing data to avoid duplicates
    collection.delete_many({})
    if data:
        collection.insert_many(data)
    print(f"Inserted {collection.count_documents({})} records into MongoDB collection '{collection_name}'.")
    client.close()

In [17]:
save_to_mongodb(extracted_data)

Inserted 1797 records into MongoDB collection 'user_history'.


## Step 6: Database Connection and Data Upload
Fetch data from MongoDB and insert into SQLite table 'user_history'

In [18]:
def mongo_to_sqlite(mongo_db='logdata', mongo_collection='user_history', sqlite_db='user_history.db'):
    # Connect to MongoDB
    client = MongoClient('mongodb://localhost:27017/')
    db = client[mongo_db]
    collection = db[mongo_collection]
    # Fetch all documents
    data = list(collection.find({}, {'_id': 0}))  # Exclude MongoDB's _id
    client.close()
    if not data:
        print('No data found in MongoDB collection.')
        return
    # Convert to DataFrame for easier handling
    df = pd.DataFrame(data)
    # Connect to SQLite
    conn = sqlite3.connect(sqlite_db)
    cursor = conn.cursor()
    # Create table with primary key constraint (email + date)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS user_history (
            email TEXT NOT NULL,
            date TEXT NOT NULL,
            PRIMARY KEY (email, date)
        )
    ''')
    # Insert data (ignore duplicates)
    for _, row in df.iterrows():
        try:
            cursor.execute('INSERT OR IGNORE INTO user_history (email, date) VALUES (?, ?)', (row['email'], row['date']))
        except Exception as e:
            print(f"Error inserting row: {row}, error: {e}")
    conn.commit()
    print(f"Inserted {cursor.execute('SELECT COUNT(*) FROM user_history').fetchone()[0]} records into SQLite table 'user_history'.")
    conn.close()

In [19]:
mongo_to_sqlite()

Inserted 1795 records into SQLite table 'user_history'.


## Step 7: Run Queries on the Database


Connect to SQLite and run required SQL queries for analysis

In [20]:
def run_sql_queries(sqlite_db='user_history.db'):
    conn = sqlite3.connect(sqlite_db)
    cursor = conn.cursor()

    print("\n--- SQL Analysis Results ---\n")

    # 1. List all unique email addresses
    print("1. Unique email addresses:")
    for row in cursor.execute('SELECT DISTINCT email FROM user_history'):
        print(row[0])
    print()

    # 2. Count the number of emails received per day
    print("2. Number of emails received per day:")
    for row in cursor.execute("SELECT substr(date, 1, 10) as day, COUNT(*) FROM user_history GROUP BY day"):
        print(f"{row[0]}: {row[1]}")
    print()

    # 3. Find the first and last email date for each email address
    print("3. First and last email date for each email address:")
    for row in cursor.execute('''
        SELECT email, MIN(date) as first_date, MAX(date) as last_date
        FROM user_history
        GROUP BY email
    '''):
        print(f"{row[0]}: First - {row[1]}, Last - {row[2]}")
    print()

    # 4. Count the total number of emails from each domain
    print("4. Total number of emails from each domain:")
    for row in cursor.execute('''
        SELECT substr(email, instr(email, '@')+1) as domain, COUNT(*)
        FROM user_history
        GROUP BY domain
        ORDER BY COUNT(*) DESC
    '''):
        print(f"{row[0]}: {row[1]}")
    print()

    # 5. Template for 10 custom questions (add your own queries here)
    print("5. Custom SQL Questions (template):")
    # 1. Top 5 most active email addresses
    print("1. Top 5 most active email addresses:")
    for row in cursor.execute('''
        SELECT email, COUNT(*) as cnt FROM user_history GROUP BY email ORDER BY cnt DESC LIMIT 5
    '''):
        print(f"{row[0]}: {row[1]}")
    print()

    # 2. Day with the highest number of emails
    print("2. Day with the highest number of emails:")
    for row in cursor.execute('''
        SELECT substr(date, 1, 10) as day, COUNT(*) as cnt FROM user_history GROUP BY day ORDER BY cnt DESC LIMIT 1
    '''):
        print(f"{row[0]}: {row[1]}")
    print()

    # 3. Number of unique domains
    print("3. Number of unique domains:")
    for row in cursor.execute('''
        SELECT COUNT(DISTINCT substr(email, instr(email, '@')+1)) FROM user_history
    '''):
        print(f"Unique domains: {row[0]}")
    print()

    # 4. Email address with the earliest email
    print("4. Email address with the earliest email:")
    for row in cursor.execute('''
        SELECT email, MIN(date) FROM user_history ORDER BY MIN(date) ASC LIMIT 1
    '''):
        print(f"{row[0]}: {row[1]}")
    print()

    # 5. Email address with the latest email
    print("5. Email address with the latest email:")
    for row in cursor.execute('''
        SELECT email, MAX(date) FROM user_history ORDER BY MAX(date) DESC LIMIT 1
    '''):
        print(f"{row[0]}: {row[1]}")
    print()

    # 6. Number of emails per domain per day (top 3 domains)
    print("6. Number of emails per domain per day (top 3 domains):")
    for row in cursor.execute('''
        SELECT substr(email, instr(email, '@')+1) as domain, substr(date, 1, 10) as day, COUNT(*) as cnt
        FROM user_history
        GROUP BY domain, day
        ORDER BY cnt DESC
        LIMIT 9
    '''):
        print(f"{row[0]} on {row[1]}: {row[2]}")
    print()

    # 7. Number of emails sent on weekends vs weekdays
    print("7. Number of emails sent on weekends vs weekdays:")
    for row in cursor.execute('''
        SELECT CASE
            WHEN strftime('%w', date) IN ('0', '6') THEN 'Weekend'
            ELSE 'Weekday'
        END as day_type, COUNT(*)
        FROM user_history
        GROUP BY day_type
    '''):
        print(f"{row[0]}: {row[1]}")
    print()

    # 8. Number of emails per month
    print("8. Number of emails per month:")
    for row in cursor.execute('''
        SELECT substr(date, 1, 7) as month, COUNT(*) FROM user_history GROUP BY month ORDER BY month
    '''):
        print(f"{row[0]}: {row[1]}")
    print()

    # 9. Email addresses that sent only one email
    print("9. Email addresses that sent only one email:")
    for row in cursor.execute('''
        SELECT email FROM user_history GROUP BY email HAVING COUNT(*) = 1
    '''):
        print(row[0])
    print()

    # 10. Most common hour for receiving emails
    print("10. Most common hour for receiving emails:")
    for row in cursor.execute('''
        SELECT substr(date, 12, 2) as hour, COUNT(*) as cnt FROM user_history GROUP BY hour ORDER BY cnt DESC LIMIT 1
    '''):
        print(f"Hour {row[0]}: {row[1]} emails")
    print()

    conn.close()

In [21]:
run_sql_queries()


--- SQL Analysis Results ---

1. Unique email addresses:
a.fish@lancaster.ac.uk
aaronz@vt.edu
ajpoland@iupui.edu
antranig@caret.cam.ac.uk
arwhyte@umich.edu
bahollad@indiana.edu
bkirschn@umich.edu
chmaurer@iupui.edu
colin.clark@utoronto.ca
csev@umich.edu
cwen@iupui.edu
david.horwitz@uct.ac.za
dlhaines@umich.edu
gbhatnag@umich.edu
ggolden@umich.edu
gjthomas@iupui.edu
gopal.ramasammycook@gmail.com
gsilver@umich.edu
hu2@iupui.edu
ian@caret.cam.ac.uk
jimeng@umich.edu
jleasia@umich.edu
jlrenfro@ucdavis.edu
john.ellis@rsmart.com
josrodri@iupui.edu
jzaremba@unicon.net
kimsooil@bu.edu
knoop@umich.edu
ktsao@stanford.edu
lance@indiana.edu
louis@media.berkeley.edu
mbreuker@loi.nl
mmmay@indiana.edu
nuno@ufp.pt
ostermmg@whitman.edu
ray@media.berkeley.edu
rjlowe@iupui.edu
sgithens@caret.cam.ac.uk
ssmail@indiana.edu
stephen.marquard@uct.ac.za
stuart.freeman@et.gatech.edu
thoppaymallika@fhda.edu
tnguyen@iupui.edu
wagnermr@iupui.edu
zach.thomas@txstate.edu
zqian@umich.edu

2. Number of emails received 